### Classic two water jugs problem

Given two jugs J1 and J2 with capacities C1 and C2, initially filled with W1 and W2.  Can you end up with exactly G1 liters in J1 and G2 liters in J2?  You're allowed the following actions: dump the contents of either jug onto the floor, or pour the contents of one jug into the other untill either the jug from which you are pouring is empty or the one you are filling is full.

### load the AIMA search.py file
We assume that search.py and any other files it needs is in this local directory or accessible in your Python environment

In [1]:
# import the AIMA search code
import search as s

### Define a subclass of AIMA search's Problem class for the Water Jug problem

In [2]:
class WJ(s.Problem):
    """
    STATE: tuple like (3,2) if jug J1 has 3 liters and J2 2 liters
    GOAL: a state except with -1 representing a 'don't care', so
      valid goals include (1,1) and (-1,2).
    PROBLEM: Specify capacities of each jug, initial state and goal """

    def __init__(self, capacities=(5,2), initial=(5,0), goal=(0,1)):
        self.capacities = capacities
        self.initial = initial
        self.goal = goal

    def __repr__(self):
        """ Returns a string representing the object """
        return f"WJ({self.capacities},{self.initial},{self.goal}"
    def goal_test(self, state):
        """ Returns True iff state is a goal state """
        G1, G2 = self.goal
        return (state[0] == G1 or G1 < 0) and \
               (state[1] == G2 or G2 < 0)

    def h(self, node):
        """ Estimate of cost of shortest path from node to a goal """
        return 0 if self.goal_test(node.state) else 1
    
    def actions(self, state):
        """ generates legal actions for state """
        (J1, J2) = state
        (C1, C2) = self.capacities
        if J1>0: yield(('dump', 1, 0))
        if J2>0: yield(('dump', 2, 0))
        if J2<C2 and J1>0: yield(('pour', 1, 2))
        if J1<C1 and J2>0: yield(('pour', 2, 1))

    def result(self, state, action):
        """Given state & action, returns successor after doing action"""
        act, arg1, arg2 = action
        (J1, J2), (C1, C2) = state, self.capacities
        if act == 'dump':
            return (0, J2) if arg1 == 1 else (J1, 0)
        elif act == 'pour':
            if arg1 == 1:
                delta = min(J1, C2-J2)
                return (J1-delta, J2+delta)
            else:
                delta = min(J2, C1-J1)
                return (J1+delta, J2-delta)

    def path_cost(self, c, state1, action, state2):
        """Cost of path from start node to state1 assuming cost c to
        get to state1 and doing action to get to state2 """
        return c + 1

### Create a problem instance

In [3]:
p = WJ((9,4), (9,4), (1,0)) 
print("Problem:", p)                                                              

Problem: WJ((9, 4),(9, 4),(1, 0)


### A node’s path is the best way to get to it from the start node, i.e., a solution
the path is a list of tuples like (action, resulting_state) from the state state to the solution found

In [4]:
def print_answer(ans):
    if ans:
        print(f"Solution with cost {ans.path_cost}: {[(node.action, node) for node in ans.path()]}")
    else:
        print("No solution found 🙁")

In [5]:
# Use breadth first graph search function. Result will be None
# if the search failed or a goal node in the search graph if successful.
print_answer(s.breadth_first_graph_search(p))

Solution with cost 5: [(None, <Node (9, 4)>), (('dump', 2, 0), <Node (9, 0)>), (('pour', 1, 2), <Node (5, 4)>), (('dump', 2, 0), <Node (5, 0)>), (('pour', 1, 2), <Node (1, 4)>), (('dump', 2, 0), <Node (1, 0)>)]


In [6]:
print_answer(s.depth_first_graph_search(p))

Solution with cost 5: [(None, <Node (9, 4)>), (('dump', 2, 0), <Node (9, 0)>), (('pour', 1, 2), <Node (5, 4)>), (('dump', 2, 0), <Node (5, 0)>), (('pour', 1, 2), <Node (1, 4)>), (('dump', 2, 0), <Node (1, 0)>)]


### AIMA python code as a function to compare searches

In [7]:
searchers=[s.breadth_first_tree_search, s.breadth_first_graph_search, s.depth_first_graph_search, s.iterative_deepening_search] 
s.compare_searchers([WJ((5,2), (5,0), (0,1))], 
                    ['SEARCH ALGORITHM', 'successors/goal tests/states generated/solution'], 
                    searchers)


SEARCH ALGORITHM             successors/goal tests/states generated/solution
breadth_first_tree_search    <  25/  26/  37/(0, >                          
breadth_first_graph_search   <   8/   9/  16/(0, >                          
depth_first_graph_search     <   5/   6/  12/(0, >                          
iterative_deepening_search   <  35/  61/  57/(0, >                          


### fin